**Basic Symptom and Disease Matching**


 1. Defining Diseases and Symptoms

In [1]:
# Defining diseases and their associated symptoms
diseases = {
    'Flu': {
        'fever': 2, 'cough': 2, 'sore throat': 1, 'runny nose': 1, 'fatigue': 1
    },
    'Malaria': {
        'fever': 3, 'chills': 3, 'sweats': 2, 'headache': 2, 'nausea': 1
    },
    'Common Cold': {
        'runny nose': 2, 'sore throat': 2, 'cough': 1, 'congestion': 1
    },
    'COVID-19': {
        'fever': 2, 'cough': 2, 'shortness of breath': 3, 'fatigue': 2
    },
    'Typhoid': {
        'fever': 3, 'abdominal pain': 3, 'fatigue': 2, 'headache': 2, 'diarrhea': 3
    },
    'Cholera': {
        'diarrhea': 3, 'vomiting': 3, 'dehydration': 2, 'leg cramps': 2
    },
    'Dysentery': {
        'diarrhea': 3, 'abdominal pain': 3, 'fever': 2, 'nausea': 1
    }
}



# We can expand this list later with more diseases and symptoms.


2. Implementing the Best-First Search algorithm

In [2]:
# A* Search function that considers severity and returns multiple diagnoses with confidence scores
def a_star_search_with_severity(symptoms_with_severity, disease_data, threshold=0.2):
    """
    A* Search: Use severity to weight the symptoms more heavily if they are severe.
    """
    results = []
    max_score = sum(max(symptom_weights.values()) for symptom_weights in disease_data.values())

    for disease, disease_symptoms in disease_data.items():
        score = 0
        for symptom, severity in symptoms_with_severity.items():
            if symptom in disease_symptoms:
                score += disease_symptoms[symptom] * severity  # Weight by severity

        confidence = score / max_score

        if confidence > threshold:
            results.append((disease, confidence))

    results.sort(key=lambda x: x[1], reverse=True)

    return results


3. installing  SpaCy for Synonym Detection

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm


^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


3. Implement Spell Correction

In [3]:
import spacy

# Load SpaCy's small English model for natural language processing
nlp = spacy.load("en_core_web_sm")

# Function to detect synonyms using word similarity from SpaCy
def detect_synonyms(symptom, valid_symptoms):
    doc1 = nlp(symptom)
    best_match = None
    highest_similarity = 0

    for valid_symptom in valid_symptoms:
        doc2 = nlp(valid_symptom)
        similarity = doc1.similarity(doc2)

        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = valid_symptom

    # Return best match if similarity is above a threshold, otherwise return the original symptom
    return best_match if highest_similarity > 0.7 else symptom  # Similarity threshold: 0.7

# Function to handle close matches for symptoms with automatic synonym detection
def correct_spelling(symptoms, valid_symptoms):
    corrected_symptoms = []
    for symptom in symptoms:
        # Use synonym detection instead of manual matching
        synonym = detect_synonyms(symptom, valid_symptoms)
        corrected_symptoms.append(synonym)
    return corrected_symptoms


4. Saving and Loading Weights

In [4]:
# Function to get feedback from the user after diagnosis
def get_user_feedback():
    feedback = input("Was the diagnosis correct? (yes/no): ").strip().lower()
    return feedback == 'yes'

# Function to adjust symptom weights based on feedback
def adjust_symptom_weights(diagnosis, symptoms, correct, disease_data):
    adjustment = 1 if correct else -0.5  # Increase for correct, decrease slightly for incorrect

    for symptom in symptoms:
        if symptom in disease_data[diagnosis]:
            disease_data[diagnosis][symptom] += adjustment
            # Ensure weights don't go below zero
            disease_data[diagnosis][symptom] = max(disease_data[diagnosis][symptom], 0)

    print(f"Updated weights for {diagnosis}: {disease_data[diagnosis]}")


5. Function to save learned data (symptom weights) to a CSV file

In [6]:
import csv

# Function to save learned data (symptom weights) to a CSV file
def save_learned_weights(disease_data, filename='learned_weights.csv'):
    with open(filename, mode='w') as file:
        writer = csv.writer(file)
        writer.writerow(["Disease", "Symptom", "Weight"])
        for disease, symptoms in disease_data.items():
            for symptom, weight in symptoms.items():
                writer.writerow([disease, symptom, weight])

# Function to load learned data from a CSV file
def load_learned_weights(filename='learned_weights.csv'):
    try:
        with open(filename, mode='r') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header
            for row in reader:
                disease, symptom, weight = row
                diseases[disease][symptom] = float(weight)
    except FileNotFoundError:
        print("No learned weights file found, using default weights.")


6. User Interaction and Main Function

In [ ]:
def get_user_input_with_severity():
    symptoms = {}

    while True:
        print("Enter your symptoms with severity (e.g., fever:3, cough:2). Severity is on a scale from 1 (mild) to 3 (severe):")
        input_str = input().lower()

        invalid_input = False  # Flag to check if any input was invalid

        # Process each symptom:severity pair
        for item in input_str.split(","):
            try:
                symptom, severity = item.split(":")
                symptoms[symptom.strip()] = int(severity.strip())
            except ValueError:
                print(f"Invalid input format for '{item.strip()}'. Please use the format 'symptom:severity'.")
                invalid_input = True
                break  # Break the loop and ask for input again

        if not invalid_input:
            break  # If all inputs are valid, exit the loop

    return symptoms


# Rule-based diagnosis for initial check
def rule_based_diagnosis(symptoms):
    if 'fever' in symptoms and 'chills' in symptoms and 'sweats' in symptoms:
        return "This might be Malaria."
    if 'fever' in symptoms and 'abdominal pain' in symptoms and 'diarrhea' in symptoms:
        return "This might be Typhoid or Cholera."

    return None


# Final main function
def main():
    print("Welcome to the Adaptive Medical Diagnosis Expert System!")

    # Load any previously learned weights
    load_learned_weights()

    # Get symptoms with severity from the user
    symptoms_with_severity = get_user_input_with_severity()

    # Extract just the symptoms for rule-based checks
    symptoms_only = symptoms_with_severity.keys()

    # Run rule-based diagnosis first
    rule_diagnosis = rule_based_diagnosis(symptoms_only)
    if rule_diagnosis:
        print(f"\nRule-based diagnosis: {rule_diagnosis}")

    # Use A* Search with severity to find multiple diagnoses
    results = a_star_search_with_severity(symptoms_with_severity, diseases)

    i = 0  # Start diagnosis index

    while True:
        if results and i < len(results):
            # Present the current best diagnosis
            best_diagnosis = results[i][0]
            confidence = results[i][1] * 100
            print(f"\nBest match: {best_diagnosis} ({confidence:.2f}% confidence)")

            # Ask for feedback
            correct = input("Was the diagnosis correct? (yes/no): ").strip().lower()

            if correct == 'yes':
                # If diagnosis is correct, adjust the weights and exit
                adjust_symptom_weights(results[i][0], symptoms_only, True, diseases)
                print(f"Updated weights for {results[i][0]}: {diseases[results[i][0]]}")
                save_learned_weights(diseases)
                break  # Exit after correct diagnosis
            elif correct == 'no':
                # Move to the next best diagnosis
                i += 1
                if i >= len(results):
                    # If no more diagnoses are available, ask for more symptoms
                    print("\nNo more diagnoses available.")
                    print("Please provide more symptoms to refine the diagnosis.")
                    new_symptoms = input("Enter additional symptoms with severity (e.g., nausea:2, fatigue:1): ").lower()
                    additional_symptoms = dict(item.split(":") for item in new_symptoms.split(","))
                    # Convert severity to integer and add to current symptoms
                    symptoms_with_severity.update({symptom.strip(): int(severity.strip()) for symptom, severity in additional_symptoms.items()})

                    # Re-run the search with updated symptoms
                    results = a_star_search_with_severity(symptoms_with_severity, diseases)
                    i = 0  # Reset diagnosis index to start again with updated symptoms
            else:
                print("Invalid input. Please type 'yes' or 'no'.")
        else:
            print("\nNo diagnosis found. Kindly visit your doctor for further consultation.")
            break

# Run the system
main()


: 